In [ ]:
# Import the required packages
import pandas as pd
import glob
import duckdb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
'''
csv_files_dustmonitoring = sorted(glob.glob(r'c:\users\sieweweiss/OneDrive - KPMG/Documents/JADS/Level 1/Enexis/data/Dustmonitoring/*.csv'), reverse=True)

df_original = pd.DataFrame()

df_original = pd.concat([pd.read_csv(
        csv_file, 
        sep=';', 
        header=[0,1,2], 
        index_col=[0,1], 
        decimal=',', 
        low_memory=False)
        for csv_file in csv_files_dustmonitoring])

df_original.dtypes
'''

In [ ]:
df_original = pd.read_pickle('df_original.pkl')

In [ ]:
df_stack = df_original
df_stack.shape

In [ ]:
df_stack = df_stack.stack(level=0)
df_stack.head()

In [ ]:
df_stacked = df_stack.reset_index(level=[0,1], names=['Time_UTC', 'Time_Local'])
df_stacked.head()

In [ ]:
df_stacked.columns = df_stacked.columns.droplevel(1)
df_stacked.head()

In [ ]:
df_rename = df_stacked.reset_index(names='Location')
df_rename.head()

In [ ]:
df_rename = df_rename.rename_axis(index='Location', columns=None)

In [ ]:
df = df_rename.reset_index(drop=True)
df.dtypes

In [ ]:
df[["Time_UTC", "Time_Local"]] = df[["Time_UTC", "Time_Local"]].apply(pd.to_datetime)

## Data Exploration & Data Prep

In [ ]:
# total number of locations in the dataset where the PM is measured 
df['Location'].nunique()

In [ ]:
df_nonNA = df.notna().groupby(df['Location']).sum().drop('Location', axis=1).reset_index()

In [ ]:
import altair as alt

chart = alt.Chart(df_nonNA).mark_bar().encode(
    alt.Y('PM10').title('Sum of non-NA PM10 count'),
    x='Location')
line = alt.Chart(df_nonNA).mark_rule(strokeDash=[10, 10]).encode(y=alt.datum(149040))


chart+line

In [ ]:
# Location IDs ordered by the number of nonNaN
df_nonNA = df.PM10.notna().groupby(df.Location).sum().reset_index().sort_values(by='PM10', ascending=False)

In [ ]:
df_na = df.PM10.isna().groupby(df.Location).sum().reset_index().sort_values(by='PM10', ascending=True)

In [ ]:
df_pm = df[['NO2', 'PM1', 'PM2.5', 'PM10', 'UFP']]

In [ ]:
sns.heatmap(df_pm.corr(), annot=True)

plt.rcParams['figure.figsize'] = (10,7)

plt.show()

In [ ]:
"""PM2.5 and PM10 are the dust measurements which are legislated by the EU and therefore the two we are most interested in. As seen above, there is a strong correlation between 
PM10 and PM2.5. Based on research by the RIVM if the PM10 meets the legislated maximum value then PM2.5 is in nearly all cases also under the maximum legislated amount. 
Therefore the focus of our project will be on PM10 and we will also drop PM2.5"""
df = df.drop(labels=['NO2', 'PM1', 'UFP', 'PM2.5'], axis=1)

In [ ]:
df['PM10'] = df['PM10'].astype('float32')
df.info()

In [ ]:
df["PM10"].describe().apply(lambda x: format(x, 'f'))

In [ ]:
df.groupby('Location')['PM10'].describe().sort_values(by='count', ascending=False)

In [ ]:
# Top 10 largest PM10 values in the dataset
df.nlargest(n=10, columns=['PM10'])

In [ ]:
# Top 10 smallest PM10 values in the dataset
df.nsmallest(n=10, columns=['PM10'])

In [ ]:
df.isnull().sum()

In [ ]:
df_rounded = df
df_rounded['Time_Local'] = pd.to_datetime(df_rounded['Time_Local']).dt.round('10min')
df_rounded['Time_UTC'] = pd.to_datetime(df_rounded['Time_UTC']).dt.round('10min')

In [ ]:
# Identify missing PM10 records
missing_count_per_location = df_rounded.groupby('Location')['PM10'].apply(lambda x: x.isna().sum())

# Group by 'Location' and calculate the total number of records
location_stats = df_rounded.groupby('Location')['PM10'].agg(['count', 'size'])

# Calculate the total number of recorded data points per location
total_recorded_data_points = location_stats['count']

# Add the missing count to the location_stats DataFrame
location_stats['missing_count'] = missing_count_per_location

# Calculate the percentage of missing data as a percentage of the total recorded values
location_stats['percentage_missing'] = (location_stats['missing_count'] / location_stats['count']) * 100

# Display the result
print(location_stats[['count', 'missing_count', 'percentage_missing']])

In [ ]:
df_rounded['Time_Local'] = pd.to_datetime(df_rounded['Time_Local'])

# Sort the DataFrame by 'Location' and 'Time_Local'
df_rounded.sort_values(['Location', 'Time_Local'], inplace=True)

# Calculate time differences between consecutive rows for each group
df_rounded['time_diff'] = df_rounded.groupby('Location')['Time_Local'].diff()

# Identify consecutive missing PM10 records greater than or equal to 2 hours
threshold_2_hours = pd.Timedelta(hours=2)
missing_data_2_hours = df_rounded[df_rounded['time_diff'] >= threshold_2_hours]

# Group by 'Location' and count consecutive missing records
missing_counts_2_hours = missing_data_2_hours.groupby('Location')['time_diff'].count()

# Get Locations with 2 hours or more consecutively missing PM10 records
locations_missing_2_hours = missing_counts_2_hours[missing_counts_2_hours >= 12].index  # 12 hours = 720 minutes / 10 minutes per measurement

print("Locations missing 2 hours or more of consecutive PM10 records:", locations_missing_2_hours)

In [ ]:
# Sort the DataFrame by 'Location' and 'Time_Local'
df_rounded.sort_values(['Location', 'Time_Local'], inplace=True)

# Identify the locations to be excluded from interpolation
exclude_locations = ['I11', 'I22', 'I28', 'I30', 'I45']

# Create a mask to filter out rows for excluded locations
mask_exclude = ~df_rounded['Location'].isin(exclude_locations)

# Apply linear interpolation for all locations except the excluded ones
df_rounded.loc[mask_exclude, 'PM10'] = df_rounded.loc[mask_exclude, 'PM10'].interpolate()

# Drop rows where 'Location' is 'I11', 'I22', 'I28', 'I30', 'I45'
locations_to_drop = ['I11', 'I22', 'I28', 'I30', 'I45']
df_rounded = df_rounded[~df_rounded['Location'].isin(locations_to_drop)]

# Drop the time_diff column
df_rounded = df_rounded.drop('time_diff', axis=1)

# Display the result
print(df_rounded)

In [ ]:
# import hourly weather data from KNMI
df_weather = pd.read_csv('KNMI Uurwaarnemingen.csv', parse_dates=['YYYYMMDD'])
df_weather.head()

In [ ]:
# Examine column names
df_weather.columns

In [ ]:
# H column contains which hour the weather conditions were measured however this is in int dtype, we need to transform it to hour format to add it to a new Datetime column which we can join to the PM10 dataset.
df_weather['Hours'] = df_weather['H'].astype(str)+':00:00'

In [ ]:
# Replace 24:00:00 with 00:00:00 for consistency
df_weather['Hours'] = df_weather['Hours'].replace('24:00:00', '00:00:00')

In [ ]:
# Create the Datetime column on which we can join the weather data and PM10 data
df_weather['Datetime'] = pd.to_datetime(df_weather['YYYYMMDD'].astype(str) + df_weather['Hours'], format='%Y-%m-%d%H:%M:%S')

In [ ]:
# we previously saw that the column names contained spaces before and after the strings, we want to remove these with strip()
df_weather.columns = df_weather.columns.str.strip()

In [ ]:
# View the table with the added columns
df_weather.head()

In [ ]:
# View the PM10 dataset
df_rounded.head()

In [ ]:
# Align datetime column format
df_rounded['Datetime'] = pd.to_datetime(df_rounded['Time_Local'].dt.to_period('H').astype(str) + ':00', format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Merge the hourly weather data and the PM10 data
df_merged = pd.merge(df_rounded, df_weather, left_on='Datetime', right_on='Datetime', how='left')

In [ ]:
df_merged.head()

In [ ]:
# Save this merged dataset to a parquet file so that it can be shared if needed and can be loaded directly without redoing the previous steps
df_merged.to_parquet('df_merged.parquet')

In [ ]:
# Load the merged dataset from the parquet file
df = pd.read_parquet('df_merged.parquet')

In [ ]:
# Create additional columns to capture day of the week, weekday/weekend data and create sine and cosine columns to capture the cyclicality of the days of the week.
df['DayOfWeek'] = df['Time_Local'].dt.dayofweek
df['Weekday_Type'] = df['Time_Local'].dt.dayofweek // 5 == 1
df['Weekday_Type'] = df['Weekday_Type'].map({True: 1, False: 0})
df['DayOfWeekSin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
df['DayOfWeekCos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)
df['Month'] = df['Time_Local'].dt.month
df['MonthSin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['MonthCos'] = np.cos(2 * np.pi * df['Month'] / 12)

In [ ]:
# Load daily weather data
df_weather_daily = pd.read_csv("KNMI Dagwaarden.csv")

In [ ]:
# Strip the spaces before/after strings in the column names
df_weather_daily.columns = df_weather_daily.columns.str.strip()

In [ ]:
df_weather_daily.info

In [ ]:
# Convert YYYYMMDD column to datetime format
df_weather_daily['YYYYMMDD'] = pd.to_datetime(df_weather_daily['YYYYMMDD'], format='%Y%m%d')

# add '_d' to all columns to distinguish between the hourly weather features already in the datatset and the hourly features we are adding next
df_weather_daily.columns = [col + '_d' for col in df_weather_daily.columns]

# Create Date column in the df to join with df_weather_daily
df['Date'] = pd.to_datetime(df['Time_Local'].dt.date, format='%Y%m%d')

# Join the two DataFrames on 'YYYYMMDD' and 'Time_Local'
result_df = pd.merge(df, df_weather_daily, left_on='Date', right_on='YYYYMMDD_d', how='left')

In [ ]:
result_df.head()

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['Hours', 'Datetime', 'YYYYMMDD', 'YYYYMMDD_d', 'Date']
result_df = result_df.drop(columns=columns_to_drop)

In [ ]:
result_df['Hour'] = result_df['Time_Local'].dt.hour

In [ ]:
result_df['hour_sin'] = np.sin(2 * np.pi * result_df['Hour'] / 24)
result_df['hour_cos'] = np.cos(2 * np.pi * result_df['Hour'] / 24)

In [ ]:
# Scope for this project is only locations in Eindhoven, these location ID's are saved in a variable
Eindhoven_locations = ['I02', 'I25', 'I14', 'I23', 'I04', 'I32', 'I37', 'I07', 'I22', 'I08', 'I28', 'I40', 'I30', 'I39', 'I12', 'I24', 'I11', 'I09', 'I36', 'I29', 'I19', 'I17']
df_eindhoven = result_df[result_df['Location'].isin(Eindhoven_locations)].copy()

In [ ]:
df_eindhoven.to_parquet('Dataset_Eindhoven.parquet')

In [ ]:
df = pd.read_parquet('Dataset_Eindhoven.parquet')

In [ ]:
# Sort the dataframe by 'Location' and 'Time_UTC' in ascending order
df = df.sort_values(by=['Location', 'Time_UTC'])

# Forward fill missing values in 'Lat' and 'Lon' columns within each location
df[['Lat', 'Lon']] = df.groupby('Location')[['Lat', 'Lon']].transform(lambda x: x.ffill())

# If there are still remaining null values after forward filling, you can fill them with the last available value
df[['Lat', 'Lon']] = df.groupby('Location')[['Lat', 'Lon']].transform(lambda x: x.fillna(method='bfill').fillna(method='ffill'))

# Check if there are any remaining null values
print(df.isnull().sum())

In [ ]:
from sklearn.model_selection import train_test_split

# Sort df by Time_Local
df = df.sort_values(by='Time_Local')

# Specify the percentage of data for training (e.g., 70% for training, 30% for testing)
train_size = 0.7

# Calculate the index to split the data
split_index = int(train_size * len(df))

# Split the data into training and testing sets
train_data = df.iloc[:split_index, :]
test_data = df.iloc[split_index:, :]

# Separate features and target variable for training set
X_train = train_data.drop(columns=['PM10', 'Time_UTC', 'Time_Local', 'Location', 'T10N', 'N', 'WW']) # Drop 'PM10' column as it is the target variable, Datetime and ID columns cant be part of X dataset and T10N, N and WW are missing too much data to include
y_train = train_data['PM10']

# Separate features and target variable for testing set
X_test = test_data.drop(columns=['PM10', 'Time_UTC', 'Time_Local', 'Location', 'T10N', 'N', 'WW']) # Drop 'PM10' column as it is the target variable, Datetime and ID columns cant be part of X dataset and T10N, N and WW are missing too much data to include
y_test = test_data['PM10']

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Evaluate the performance
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

# Print evaluation metrics
print(f'Mean Squared Error (Random Forest): {mse_rf:.2f}')
print(f'Mean Absolute Error (Random Forest): {mae_rf:.2f}')
print(f'R-squared (Random Forest): {r2_rf:.2f}')
print(f'Root Mean Squared Error (Random Forest): {rmse_rf:.2f}')

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# Create an Extra Trees Regressor model
extra_trees_model = ExtraTreesRegressor(random_state=42)

# Train the model
extra_trees_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_extra_trees = extra_trees_model.predict(X_test)

# Evaluate the model performance
mse_extra_trees = mean_squared_error(y_test, y_pred_extra_trees)
mae_extra_trees = mean_absolute_error(y_test, y_pred_extra_trees)
r2_extra_trees = r2_score(y_test, y_pred_extra_trees)
rmse_extra_trees = np.sqrt(mse_extra_trees)

# Print the evaluation metrics
print(f'Mean Squared Error (Extra Trees): {mse_extra_trees:.2f}')
print(f'Mean Absolute Error (Extra Trees): {mae_extra_trees:.2f}')
print(f'R-squared (Extra Trees): {r2_extra_trees:.2f}')
print(f'Root Mean Squared Error (Extra Trees): {rmse_extra_trees:.2f}')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create a Decision Tree Regressor model
decision_tree_model = DecisionTreeRegressor(random_state=42)

# Train the model
decision_tree_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_decision_tree = decision_tree_model.predict(X_test)

# Evaluate the model performance
mse_decision_tree = mean_squared_error(y_test, y_pred_decision_tree)
mae_decision_tree = mean_absolute_error(y_test, y_pred_decision_tree)
r2_decision_tree = r2_score(y_test, y_pred_decision_tree)
rmse_decision_tree = np.sqrt(mse_decision_tree)

# Print the evaluation metrics
print(f'Mean Squared Error (Decision Tree): {mse_decision_tree:.2f}')
print(f'Mean Absolute Error (Decision Tree): {mae_decision_tree:.2f}')
print(f'R-squared (Decision Tree): {r2_decision_tree:.2f}')
print(f'Root Mean Squared Error (Decision Tree): {rmse_decision_tree:.2f}')